In [1]:
import numpy as np
import uproot3
import matplotlib.pyplot as plt

In [2]:
%matplotlib widget

In [3]:
infile_bitumen = 'data/largedrum_onlybitumen_dense_newmetrics_3cmVoxel_April2021.discriminator.root'

In [4]:
fu = uproot3.open(infile_bitumen)
histogram_bitumen = fu['histMedianMetric'].numpy()[0]
bins_x, bins_y, bins_z = fu['histMedianMetric;1'].numpy()[1][0]
shape_x, shape_y, shape_z = histogram_bitumen.shape

In [5]:
infile_hydrogen = 'data/largedrum_onlyhydrogen_dense_newmetrics_3cmVoxel_April2021.discriminator.root'

In [6]:
fu = uproot3.open(infile_hydrogen)
histogram_hydrogen = fu['histMedianMetric'].numpy()[0]
bins_x, bins_y, bins_z = fu['histMedianMetric;1'].numpy()[1][0]
shape_x, shape_y, shape_z = histogram_hydrogen.shape

In [7]:
bincentre_x = []
for i, _ in enumerate(bins_x[:-1]):
    #print(i, bins_x[i], bins_x[i+1],(bins_x[i]+bins_x[i+1])/2 )
    bincentre_x.append((bins_x[i]+bins_x[i+1])/2)
bincentre_x = np.asarray(bincentre_x)

In [8]:
bincentre_y = []
for i, _ in enumerate(bins_y[:-1]):
    bincentre_y.append((bins_y[i]+bins_y[i+1])/2)
bincentre_y = np.asarray(bincentre_y)

In [9]:
bincentre_z = []
for i, _ in enumerate(bins_z[:-1]):
    bincentre_z.append((bins_z[i]+bins_z[i+1])/2)
bincentre_z = np.asarray(bincentre_z)

In [144]:
def select_histogram_coords(histogram, x_min, x_max, y_min, y_max, z_min, z_max):
    
    sel_x = np.where(np.logical_and(bincentre_x>=x_min, bincentre_x<=x_max))[0]#[:-1]
    sel_y = np.where(np.logical_and(bincentre_y>=y_min, bincentre_y<=y_max))[0]#[:-1]
    sel_z = np.where(np.logical_and(bincentre_z>=z_min, bincentre_z<=z_max))[0]#[:-1]
    
    print(sel_x.shape, sel_y.shape, sel_z.shape)

    bincentre_sel_x = bincentre_x[sel_x]
    bincentre_sel_y = bincentre_y[sel_y] 
    bincentre_sel_z = bincentre_z[sel_z]
    #print(sel_x, bincentre_x[sel_x],bincentre_sel_x)
    xx, yy, zz = np.meshgrid(sel_x, sel_y, sel_z, indexing='ij')
    selected_histogram = np.copy(histogram[xx, yy, zz])
    
    return selected_histogram, (bincentre_sel_x, bincentre_sel_y, bincentre_sel_z)

In [147]:
_ = select_histogram_coords(histogram_bitumen, -440, 440, -300, 300, -300, 300)

(30,) (20,) (19,)


In [145]:
def select_histogram_cylinder(histogram, sel_bins, cut_on_yz):
    sbx, sby, sbz = sel_bins
    sel_y =  np.where(np.sqrt(sby**2+sbz**2)<=cut_on_yz)[0]
    sel_z = sel_y
    sel_x = np.where(sbx>-1000)[0] 
    sby = sby[sel_y] 
    sbz = sbz[sel_z]
    #print(sel_x, bincentre_x[sel_x],bincentre_sel_x)
    xx, yy, zz = np.meshgrid(sel_x, sel_y, sel_z, indexing='ij')
    selected_histogram = np.copy(histogram[xx, yy, zz])
    bincentre_sel_x = sbx
    bincentre_sel_y = sby
    bincentre_sel_z = sbz
    return selected_histogram, (bincentre_sel_x, bincentre_sel_y, bincentre_sel_z)

In [146]:
sel_bt_hist, sel_bt_hist_bins = select_histogram_coords(histogram_bitumen, -1000, 1000, -500, 500, -500, 500)
sel_h_hist, sel_h_hist_bins = select_histogram_coords(histogram_hydrogen, -1000, 1000, -500, 500, -500, 500)

(66,) (33,) (33,)
(66,) (33,) (33,)


In [56]:
sel_cyl_bt_hist, sel_cyl_bt_bins = select_histogram_cylinder(sel_bt_hist, sel_bt_hist_bins,300 )
sel_cyl_h_hist, sel_cyl_h_bins = select_histogram_cylinder(sel_h_hist, sel_h_hist_bins,300 )


In [123]:
def cut_regions(histo):
    bitumen_left_hist, bins_left_bitumen = select_histogram_coords(histo, -440, -200, -300, 300, -300, 300)
    bitumen_right_hist, bins_right_bitumen = select_histogram_coords(histo, 200, 440, -300, 300, -300, 300)
    bubble_hist, bins_bubble = select_histogram_coords(histo, -100, 100, -100, 100, -100, 100)
    drum_hist, bins_drum = select_histogram_coords(histo, -440, 440, -340, 340, -340, 340)
    return bitumen_left_hist, bins_left_bitumen, bitumen_right_hist, bins_right_bitumen, bubble_hist, bins_bubble, drum_hist, bins_drum

In [124]:
bt_left_hist, bins_left_bt, bt_right_hist, bins_right_bt, bt_centre_hist, bins_bt_centre, drum_bt_hist, bins_drum_bt = cut_regions(histogram_bitumen)

In [125]:
bins = np.linspace(10,15, 100)
plt.figure()
plt.ylim(0,1200)
plt.hist(drum_bt_hist[drum_bt_hist>0].ravel(), bins=bins, label='all data - drum')
plt.hist(bt_left_hist.ravel(), alpha=.7,bins=bins, label='"bitumen" left')
plt.hist(bt_right_hist.ravel(),alpha=.7, bins=bins, label='"bitumen" right')
plt.hist(bt_centre_hist.ravel(), bins=bins, label='"bubble"')
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [126]:
h_left_hist, bins_left_h, h_right_hist, bins_right_h, h_centre_hist, bins_h_centre, drum_h_hist, bins_drum_h = cut_regions(histogram_hydrogen)

In [127]:
bins = np.linspace(10,15, 100)
plt.figure()
plt.ylim(0,1200)
plt.hist(drum_h_hist[drum_h_hist>0].ravel(), bins=bins, label='all data - drum')
plt.hist(h_left_hist.ravel(), alpha=.7,bins=bins, label='"bitumen" left')
plt.hist(h_right_hist.ravel(),alpha=.7, bins=bins, label='"bitumen" right')
plt.hist(h_centre_hist.ravel(), bins=bins, label='"bubble"')
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [128]:
def distance_and_metric(histogram, indices, bins):
    distance_yz = []
    distance_xyz = []
    distance_z = []
    median = []
    indices_x, indices_y, indices_z = indices
    
    bins_x, bins_y, bins_z = bins
    for x, y, z in zip(indices_x, indices_y, indices_z):
        if np.sqrt(bins_y[y]**2 + bins_z[z]**2)<=300:
            #print(x,y,z,  np.sqrt(bins_x[x]**2+bins_y[y]**2), np.sqrt(bins_x[x]**2+bins_y[y]**2+bins_z[z]**2), histogram[x,y,z])
            distance_yz.append(np.sqrt(bins_y[y]**2 + bins_z[z]**2))
            distance_xyz.append(np.sqrt(bins_x[x]**2 + bins_y[y]**2 + bins_z[z]**2))
            distance_z.append(bins_z[z])
            median.append(histogram[x,y,z])
    distance_z = np.asarray(distance_z)
    distance_yz = np.asarray(distance_yz)
    distance_xyz = np.asarray(distance_xyz)
    median = np.asarray(median)
    return distance_yz, median, distance_z, distance_xyz

In [129]:
def distance_x_axis(histogram, indices, bins):
    distance_yz = []
    median = []
    indices_x, indices_y, indices_z = indices
    
    bins_x, bins_y, bins_z = bins
    for x, y, z in zip(indices_x, indices_y, indices_z):
        if ((bins_y[y]**2 + bins_z[z]**2)<=290**2) & (bins_x[x]>-400) & (bins_x[x]<=400):
            
            distance_yz.append(np.sqrt(bins_y[y]**2 + bins_z[z]**2))
            median.append(histogram[x,y,z])
    distance_yz = np.asarray(distance_yz)
    median = np.asarray(median)
    return distance_yz, median

In [130]:
def distance_centre(histogram, indices, bins):
    distance_xyz = []
    median = []
    indices_x, indices_y, indices_z = indices
    bins_x, bins_y, bins_z = bins
    for x, y, z in zip(indices_x, indices_y, indices_z):
        if ((bins_y[y]**2 + bins_z[z]**2)<=290**2) & (bins_x[x]>-400) & (bins_x[x]<=400):
            distance_xyz.append(np.sqrt(bins_x[x]**2 + bins_y[y]**2 + bins_z[z]**2))
            median.append(histogram[x,y,z])
    distance_xyz = np.asarray(distance_xyz)
    median = np.asarray(median)
    return distance_xyz, median

In [131]:
def distance_z(histogram, indices, bins):
    distance_z = []
    median = []
    indices_x, indices_y, indices_z = indices
    bins_x, bins_y, bins_z = bins
    for x, y, z in zip(indices_x, indices_y, indices_z):
        if np.sqrt(bins_y[y]**2 + bins_z[z]**2)<=300:
            #print(x,y,z,  np.sqrt(bins_x[x]**2+bins_y[y]**2), np.sqrt(bins_x[x]**2+bins_y[y]**2+bins_z[z]**2), histogram[x,y,z])
            distance_z.append(bins_z[z])
            median.append(histogram[x,y,z])
    distance_z = np.asarray(distance_z)
    median = np.asarray(median)
    return distance_z, median

In [132]:
bt_left_dyz, bt_left_dyz_med = distance_x_axis(bt_left_hist, np.where(bt_left_hist>0), bins_left_bt)
bt_left_dxyz, bt_left_dxyz_med = distance_centre(bt_left_hist, np.where(bt_left_hist>0), bins_left_bt)
bt_left_dz, bt_left_dyz_med = distance_z(bt_left_hist, np.where(bt_left_hist>0), bins_left_bt)

In [133]:
bt_right_dyz, bt_right_dyz_med = distance_x_axis(bt_right_hist, np.where(bt_right_hist>0), bins_right_bt)
bt_right_dxyz, bt_right_dxyz_med = distance_centre(bt_right_hist, np.where(bt_right_hist>0), bins_right_bt)
bt_right_dz, bt_right_dyz_med = distance_z(bt_right_hist, np.where(bt_right_hist>0), bins_right_bt)

In [134]:
bt_centre_dyz, bt_centre_dyz_med = distance_x_axis(bt_centre_hist, np.where(bt_centre_hist>0), bins_bt_centre)
bt_centre_dxyz, bt_centre_dxyz_med = distance_centre(bt_centre_hist, np.where(bt_centre_hist>0), bins_bt_centre)
bt_centre_dz, bt_centre_dz_med = distance_z(bt_centre_hist, np.where(bt_centre_hist>0), bins_bt_centre)

In [135]:
bt_drum_dyz, bt_drum_dyz_med = distance_x_axis(drum_bt_hist, np.where(drum_bt_hist>0), bins_drum_bt)
bt_drum_dxyz, bt_drum_dxyz_med = distance_centre(drum_bt_hist, np.where(drum_bt_hist>0), bins_drum_bt)
bt_drum_dz, bt_drum_dz_med = distance_z(drum_bt_hist, np.where(drum_bt_hist>0), bins_drum_bt)

bt_left_dist, bt_left_med, bt_left_dz, bt_left_dxyz = distance_and_metric(bt_left_hist, np.where(bt_left_hist>0), bins_left_bt)
bt_right_dist, bt_right_med, bt_right_dz, bt_right_dxyz = distance_and_metric(bt_right_hist, np.where(bt_right_hist>0), bins_right_bt)
bt_air_dist, bt_air_med, bt_air_dz, bt_air_dxyz = distance_and_metric(bt_centre_hist,np.where(bt_centre_hist>0), bins_bt_centre)

bt_all_data_d, bt_all_data_med, bt_all_data_dz, bt_all_data_dxyz = distance_and_metric(histogram_bitumen, np.where(histogram_bitumen>0), (bincentre_x, bincentre_y, bincentre_z))

bt_all_data_drum_d, bt_all_data_drum_med, bt_all_data_drum_dz, bt_all_data_drum_dxyz = distance_and_metric(drum_bt_hist, np.where(drum_bt_hist>=0),
                                                                                               bins_drum_bt)

In [136]:
h_left_dyz, h_left_dyz_med = distance_x_axis(h_left_hist, np.where(h_left_hist>0), bins_left_h)
h_left_dxyz, h_left_dxyz_med = distance_centre(h_left_hist, np.where(h_left_hist>0), bins_left_h)
h_left_dz, h_left_dyz_med = distance_z(h_left_hist, np.where(h_left_hist>0), bins_left_h)

In [137]:
h_right_dyz, h_right_dyz_med = distance_x_axis(h_right_hist, np.where(h_right_hist>0), bins_right_h)
h_right_dxyz, h_right_dxyz_med = distance_centre(h_right_hist, np.where(h_right_hist>0), bins_right_h)
h_right_dz, h_right_dyz_med = distance_z(h_right_hist, np.where(h_right_hist>0), bins_right_h)

In [138]:
h_centre_dyz, h_centre_dyz_med = distance_x_axis(h_centre_hist, np.where(h_centre_hist>0), bins_h_centre)
h_centre_dxyz, h_centre_dxyz_med = distance_centre(h_centre_hist, np.where(h_centre_hist>0), bins_h_centre)
h_centre_dz, h_centre_dz_med = distance_z(h_centre_hist, np.where(h_centre_hist>0), bins_h_centre)

In [139]:
h_drum_dyz, h_drum_dyz_med = distance_x_axis(drum_h_hist, np.where(drum_h_hist>0), bins_drum_h)
h_drum_dxyz, h_drum_dxyz_med = distance_centre(drum_h_hist, np.where(drum_h_hist>0), bins_drum_h)
h_drum_dz, h_drum_dz_med = distance_z(drum_h_hist, np.where(drum_h_hist>0), bins_drum_h)

In [140]:
h_cyl_dyz, h_cyl_dyz_med = distance_x_axis(sel_cyl_h_hist, np.where(sel_cyl_h_hist>0), sel_cyl_h_bins)
h_cyl_dxyz, h_cyl_dxyz_med = distance_centre(sel_cyl_h_hist, np.where(sel_cyl_h_hist>0), sel_cyl_h_bins)

bt_cyl_dyz, bt_cyl_dyz_med = distance_x_axis(sel_cyl_bt_hist, np.where(sel_cyl_bt_hist>0), sel_cyl_bt_bins)
bt_cyl_dxyz, bt_cyl_dxyz_med = distance_centre(sel_cyl_bt_hist, np.where(sel_cyl_bt_hist>0), sel_cyl_bt_bins)



h_left_dist, h_left_med, h_left_dz, h_left_dxyz = distance_and_metric(h_left_hist, np.where(h_left_hist>0), bins_left_h)
h_right_dist, h_right_med, h_right_dz, h_right_dxyz = distance_and_metric(h_right_hist, np.where(h_right_hist>0), bins_right_h)
h_air_dist, h_air_med, h_air_dz, h_air_dxyz = distance_and_metric(h_centre_hist,np.where(h_centre_hist>0), bins_h_centre)
h_all_data_d, h_all_data_med, h_all_data_dz, h_all_data_dxyz = distance_and_metric(histogram_hydrogen, np.where(histogram_hydrogen>0), (bincentre_x, bincentre_y, bincentre_z))
h_all_data_drum_d, h_all_data_drum_med, h_all_data_drum_dz, h_all_data_drum_dxyz = distance_and_metric(drum_h_hist, np.where(drum_h_hist>=0), bins_drum_h)
 

In [141]:
plt.figure(figsize=(10,6))
plt.scatter(bt_centre_dyz, bt_centre_dyz_med, c='blue', s=10, marker='s', alpha=.5, label='bitumen; x, y, z in (-100;100)')
plt.scatter(h_centre_dyz, h_centre_dyz_med, c='crimson', s=10, marker='s', alpha=.4, label='hydrogen: x, y, z in (-100;100)')


plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
#plt.ylim(9.5,13)
#plt.ylim(0,15)
#plt.yticks(np.arange(0,15,1))
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [142]:
plt.figure(figsize=(10,6))
plt.scatter(bt_drum_dyz_med, bt_drum_dyz, c='blue', s=5, marker='s', alpha=.5, label='drum bitumen')
#plt.scatter(h_drum_dyz_med, h_drum_dyz,  c='crimson', s=10, marker='s', alpha=.4, label='drum hydrogen')


plt.axvline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
#plt.ylim(9.5,13)
plt.xlim(0,15)
#plt.yticks(np.arange(0,15,1))
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
plt.figure(figsize=(10,6))
#plt.scatter(bt_drum_dyz_med, bt_drum_dyz, c='blue', s=10, marker='s', alpha=.5, label='drum bitumen')
plt.scatter(h_drum_dyz_med, h_drum_dyz,  c='crimson', s=2, marker='s', alpha=.4, label='drum hydrogen')


plt.axvline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
#plt.ylim(9.5,13)
plt.xlim(0,15)
#plt.yticks(np.arange(0,15,1))
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
plt.figure(figsize=(10,6))
plt.scatter(bt_cyl_dyz_med, bt_cyl_dyz, c='blue', s=10, marker='s', alpha=.5, label='drum bitumen')
#plt.scatter(h_drum_dyz_med, h_drum_dyz,  c='crimson', s=10, marker='s', alpha=.4, label='drum hydrogen')


plt.axvline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
#plt.ylim(9.5,13)
plt.xlim(0,15)
#plt.yticks(np.arange(0,15,1))
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
plt.figure(figsize=(10,6))
#plt.scatter(bt_drum_dyz_med, bt_drum_dyz, c='blue', s=10, marker='s', alpha=.5, label='drum bitumen')
plt.scatter(h_cyl_dyz_med, h_cyl_dyz,  c='crimson', s=10, marker='s', alpha=.4, label='drum hydrogen')


plt.axvline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
#plt.ylim(9.5,13)
plt.xlim(0,15)
#plt.yticks(np.arange(0,15,1))
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [148]:
plt.figure(figsize=(10,6))
plt.scatter (bt_drum_dxyz_med,bt_drum_dxyz, c='blue', s=5, marker='s', alpha=.5, label='alldata drum bitumen')
#plt.scatter (h_drum_dxyz_med,h_drum_dxyz, c='crimson', s=5, marker='s', alpha=.3, label='alldata drum hydrogen')
plt.axvline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
#plt.ylim(9.5,13)
plt.xlim(0,15)
#plt.yticks(np.arange(0,15,1))
plt.xlabel('distance from axes origin (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
plt.figure(figsize=(10,6))
plt.scatter (bt_drum_dz, bt_drum_dz_med, c='blue', s=5, marker='s', alpha=.5, label='alldata drum bitumen')
plt.scatter (h_drum_dz , h_drum_dz_med, c='crimson', s=5, marker='s', alpha=.3, label='alldata drum hydrogen')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
#plt.ylim(9.5,13)
#plt.xlim(0,15)
#plt.yticks(np.arange(0,15,1))
plt.xlabel('z coord (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …